# PyCap runs for Little Plover River - based on Wisconsin DNR strategy

This notebook assumes that you have already created a excel file with all the starting information. This script will convert that `.xlsx` file into a `.yml`. 


In [ ]:
import yaml
import pandas as pd
import numpy as np
from pathlib import Path
import sys
sys.path.append('../scripts')
from wdnr_pycap import Excel2YML
import matplotlib.pyplot as plt
import os, platform
from pycap.analysis_project import Project
import geopandas as gpd
from shapely.geometry import Point
from  datetime import date


In [ ]:
#### Path to excel input file. Note can use absolute or relative path.
pycap_inputs_excel = Path("../Inputs/LPR_Prepped_IDW.xlsx")

#### PyCap Run Name is what all your outputs will have as a name. 
pycap_run_name = "LPR_Redux"

#### Base directory for runs
parent_run_path = Path("../pycap_runs")

#### base run directory
base_run_path = parent_run_path / "pycap_base"

Reviewer_Name = "MNF"


In [ ]:
if not base_run_path.exists():
    base_run_path.mkdir(parents=True)

### Create YML from Excel
##### Created with a function

In [ ]:
project_dict, stream_dict, well_dict, ddresp_dict, combdict = Excel2YML(pycap_inputs_excel,pycap_run_name, base_run_path)
     

### Now run the base model with `pycap`

In [ ]:
yml_filename = f"{pycap_run_name}.yml"

#### We create a `Project`, then report the responses and finally write the responses to an external CSV file

In [ ]:
ap = Project(base_run_path /  yml_filename)
ap.report_responses()
ap.write_responses_csv()

#### Now let's have a look at the main results files
##### First the summary base depletion

In [ ]:
depl = pd.read_csv(
    base_run_path / f"output/{pycap_run_name}.table_report.base_stream_depletion.csv", index_col = 0
)
depl

In [ ]:
all_time_series = pd.read_csv(
    base_run_path / f"output/{pycap_run_name}.table_report.all_ts.csv".format(pycap_run_name), index_col=0
)

cum_depl = all_time_series.sum(axis=1)

fig,ax = plt.subplots(nrows=1, ncols=1)
ax.plot(all_time_series, alpha=0.6, c='k', lw=0.2)
ax.plot(cum_depl, c='r')
ax.set_title('Cumulative Depl, LPR at Hoover,Base Case Analysis')
ax.set_xlabel("Days")
ax.set_ylabel('Depletion, cfs')

In [ ]:
LPR_baseline_flow = 8.6

In [ ]:
all_time_series= LPR_baseline_flow-all_time_series
cum_depl = LPR_baseline_flow - cum_depl

In [ ]:
fig,ax = plt.subplots(nrows=1, ncols=1)
ax.plot(cum_depl, c='r')
ax.set_title('Streamflow, LPR at Hoover,Base Case Analysis')
ax.set_xlabel("Days")
ax.set_ylabel('Streamflow, cfs')